In [ ]:
!pip install torchaudio
import torchaudio
import torch

     |████████████████████████████████| 1.9MB 6.5MB/s 


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans

In [ ]:
import numpy as np
import librosa
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
import os
from pathlib import Path

In [ ]:
!gdown https://drive.google.com/uc?id=1iVEgEKm1tS1upClYWih6ikEiwd8mh0Vm

Downloading...
From: https://drive.google.com/uc?id=1iVEgEKm1tS1upClYWih6ikEiwd8mh0Vm
To: /content/voice_print.zip
2.01GB [00:17, 113MB/s]


In [ ]:
!unzip -q voice_print

In [ ]:
data = pd.read_csv("/content/voice_print.csv")

In [ ]:
data_test = data.loc[data['mode'] == 'test']

In [ ]:
X_test, y_test = data_test.voiceprint, data_test.label

In [ ]:
data_train = data.loc[data['mode'] == 'train']

In [ ]:
data_valid = data.loc[data['mode'] == 'valid']

In [ ]:
X_valid, y_valid = data_valid.voiceprint, data_valid.label

In [ ]:
X_train, y_train = data_train.voiceprint, data_train.label

In [ ]:
def get_mfcc(amplitudes, labels):
  X = []
  y = []
  for amp in amplitudes:
    amp_t = torch.from_numpy(np.array(amp[1:-1].split(","), dtype=np.float32)).view(1, 8000)
    mfcc = torchaudio.transforms.MFCC(8000, 20, log_mels=False,
                                          melkwargs={'n_fft': 512, 'n_mels': 20, 'win_length': 200, 'hop_length': 80,
                                                     'f_min': 70.0, 'f_max': 4000.0})(amp_t)
    
    X.append(mfcc.numpy())
  for label in labels:
    y.append(label)
    # y.append(label)
    # y.append(label)
  return X, y

In [ ]:
X_train_mfcc, y_train_mfcc  = get_mfcc(X_train, y_train)
X_valid_mfcc, y_valid_mfcc = get_mfcc(X_valid, y_valid)

In [ ]:
# X_train_mfcc = torch.cat(X_train_mfcc)
# X_valid_mfcc = torch.cat(X_valid_mfcc)
# y_train_mfcc = torch.LongTensor(y_train_mfcc)
# y_valid_mfcc = torch.LongTensor(y_valid_mfcc)

(1, 120, 101)

In [ ]:
X_test_mfcc, y_test_mfcc = get_mfcc(X_test, y_test)
X_test_mfcc = np.array(X_test_mfcc)
a = X_test_mfcc.shape[3]
b = X_test_mfcc.shape[2]
X_test_mfcc = X_test_mfcc.reshape((1384, a*b))
y_test_mfcc = np.array(y_test_mfcc)

In [ ]:
def generate_batches(X, y, batch_size=64):
    assert len(X) == len(y)
    np.random.seed(42)
    X = np.array(X)
    y = np.array(y)
    perm = np.random.permutation(len(X))
    for start in range(0, (len(X)//batch_size)*batch_size, batch_size):
      end = min(start + batch_size, len(X))
      batch_idx = perm[start:end]
      yield X[batch_idx], y[batch_idx]

# SGD SVM

In [ ]:
sgd_svm = SGDClassifier()

In [ ]:
batch_size = 64
epochs = 50

In [ ]:
for epoch in range(epochs):
  for X_b, y_b in generate_batches(X_train_mfcc, y_train_mfcc, batch_size):
    a = X_b.shape[3]
    b = X_b.shape[2]
    X_b = X_b.reshape((batch_size, a*b))
    sgd_svm.partial_fit(X_b, y_b, classes=np.unique(y_test_mfcc))
  for X_b, y_b in generate_batches(X_valid_mfcc, y_valid_mfcc, batch_size):
    a = X_b.shape[3]
    b = X_b.shape[2]
    X_b = X_b.reshape((batch_size, a*b))
    sgd_svm.partial_fit(X_b, y_b, classes=np.unique(y_test_mfcc)) 
  print(epoch, sgd_svm.score(X_test_mfcc, y_test_mfcc))

0 0.444364161849711
1 0.5021676300578035
2 0.5794797687861272
3 0.5036127167630058
4 0.6380057803468208
5 0.6336705202312138
6 0.7117052023121387
7 0.786849710982659
8 0.7377167630057804
9 0.7911849710982659
10 0.6958092485549133
11 0.6965317919075145
12 0.7073699421965318
13 0.8735549132947977
14 0.7817919075144508
15 0.8005780346820809
16 0.880057803468208
17 0.7803468208092486
18 0.8569364161849711
19 0.9060693641618497
20 0.8078034682080925
21 0.8460982658959537
22 0.880057803468208
23 0.8706647398843931
24 0.903179190751445
25 0.8995664739884393
26 0.846820809248555
27 0.9125722543352601
28 0.9082369942196532
29 0.8923410404624278
30 0.9060693641618497
31 0.9313583815028902
32 0.944364161849711
33 0.953757225433526
34 0.9617052023121387
35 0.9465317919075145
36 0.9263005780346821
37 0.9458092485549133
38 0.9776011560693642
39 0.9385838150289018
40 0.9544797687861272
41 0.9653179190751445
42 0.9660404624277457
43 0.9104046242774566
44 0.9667630057803468
45 0.976878612716763
46 0.92

In [ ]:
e = []
p = []
for i in range(len(y_test)):
  pred = sgd_svm.predict([X_test_mfcc[i]])
  if pred!=y_test_mfcc[i]:
    p.append(pred)
    e.append(y_test_mfcc[i])

In [ ]:
p

[array([47]),
 array([189]),
 array([109]),
 array([117]),
 array([62]),
 array([183]),
 array([117]),
 array([99]),
 array([135]),
 array([62]),
 array([117]),
 array([135]),
 array([135]),
 array([175]),
 array([135]),
 array([117]),
 array([105]),
 array([117]),
 array([117]),
 array([29]),
 array([99]),
 array([183]),
 array([105]),
 array([117]),
 array([168]),
 array([171]),
 array([109]),
 array([29]),
 array([102]),
 array([117]),
 array([189]),
 array([32]),
 array([117]),
 array([117]),
 array([117]),
 array([1]),
 array([127]),
 array([139]),
 array([135]),
 array([117]),
 array([135]),
 array([183]),
 array([102]),
 array([62]),
 array([109]),
 array([117]),
 array([29]),
 array([135]),
 array([117]),
 array([183]),
 array([135]),
 array([102]),
 array([89]),
 array([10]),
 array([189]),
 array([135]),
 array([117]),
 array([117]),
 array([62]),
 array([117]),
 array([117]),
 array([62]),
 array([83]),
 array([117]),
 array([117]),
 array([117]),
 array([117]),
 array([117]

In [ ]:
p = []
for i in range(len(y_test)):
  pred = sgd_svm.predict([X_test_mfcc[i]])
  p.append(pred)
p[-40:-1]

[array([197]),
 array([198]),
 array([198]),
 array([198]),
 array([111]),
 array([199]),
 array([199]),
 array([199]),
 array([200]),
 array([200]),
 array([200]),
 array([200]),
 array([200]),
 array([200]),
 array([201]),
 array([201]),
 array([201]),
 array([201]),
 array([201]),
 array([201]),
 array([202]),
 array([202]),
 array([202]),
 array([202]),
 array([202]),
 array([203]),
 array([203]),
 array([203]),
 array([203]),
 array([204]),
 array([204]),
 array([204]),
 array([111]),
 array([205]),
 array([205]),
 array([60]),
 array([56]),
 array([135]),
 array([93])]

In [ ]:
y_test_mfcc[-10:-1]

array([204, 204, 204, 205, 205, 205, 205, 205, 205])

In [ ]:
p[-10:-1]

[array([204]),
 array([204]),
 array([204]),
 array([127]),
 array([205]),
 array([141]),
 array([56]),
 array([205]),
 array([93])]

In [ ]:
a = X_test_mfcc.shape[1]
b = X_test_mfcc.shape[2]
X_test_mfcc = X_test_mfcc.reshape((4152, a*b))
y_test = np.array(y_test)

0.34344894026974954

In [ ]:
import joblib

In [ ]:
filename = 'svm_sgd_70_97_4935s.sav'
joblib.dump(sgd_svm, filename)

['svm_sgd_70_97_4935s.sav']

# Passive Agressive Classifier

In [ ]:
from sklearn.linear_model import PassiveAggressiveClassifier

In [ ]:
pac = PassiveAggressiveClassifier(C=0.2, loss='hinge', warm_start=True, average=True)

In [ ]:
batch_size = 64
epochs = 25

In [ ]:
for epoch in range(epochs):
  for X_b, y_b in generate_batches(X_train_mfcc, y_train_mfcc, batch_size):
    a = X_b.shape[3]
    b = X_b.shape[2]
    X_b = X_b.reshape((batch_size, a*b))
    pac.partial_fit(X_b, y_b, classes=np.unique(y_test_mfcc))
  for X_b, y_b in generate_batches(X_valid_mfcc, y_valid_mfcc, batch_size):
    a = X_b.shape[3]
    b = X_b.shape[2]
    X_b = X_b.reshape((batch_size, a*b))
    pac.partial_fit(X_b, y_b, classes=np.unique(y_test_mfcc)) 
  print(epoch, pac.score(X_test_mfcc, y_test_mfcc))

0 0.8735549132947977
1 0.8786127167630058
2 0.8807803468208093
3 0.8822254335260116
4 0.8858381502890174
5 0.8872832369942196
6 0.8908959537572254
7 0.8916184971098265
8 0.8937861271676301
9 0.8959537572254336
10 0.8959537572254336
11 0.8988439306358381
12 0.9002890173410405
13 0.903179190751445
14 0.9039017341040463
15 0.9060693641618497
16 0.9067919075144508
17 0.9096820809248555
18 0.911849710982659
19 0.9125722543352601
20 0.9132947976878613
21 0.9147398843930635
22 0.9161849710982659
23 0.9161849710982659


KeyboardInterrupt: ignored

In [ ]:
pac.score(X_test_mfcc, y_test_mfcc)

0.9884393063583815

In [ ]:
e = []
p = []
for i in range(len(y_test)):
  pred = pac.predict([X_test_mfcc[i]])
  if pred!=y_test_mfcc[i]:
    p.append(pred)
    e.append(y_test_mfcc[i])
p, e

([array([135]),
  array([111]),
  array([109]),
  array([119]),
  array([111]),
  array([135]),
  array([183]),
  array([87]),
  array([109]),
  array([127]),
  array([64]),
  array([132]),
  array([56]),
  array([135]),
  array([135]),
  array([118])],
 [24, 27, 66, 102, 105, 111, 122, 174, 191, 205, 205, 205, 205, 205, 205, 205])

In [ ]:
p = []
for i in range(len(y_test)):
  pred = pac.predict([X_test_mfcc[i]])
  p.append(pred)
p[-10:-1]

[array([204]),
 array([204]),
 array([204]),
 array([127]),
 array([64]),
 array([132]),
 array([56]),
 array([135]),
 array([135])]

([array([135]),
  array([135]),
  array([135]),
  array([109]),
  array([135]),
  array([119]),
  array([135]),
  array([135]),
  array([135]),
  array([135]),
  array([109]),
  array([135]),
  array([135]),
  array([135]),
  array([127]),
  array([64]),
  array([132]),
  array([56]),
  array([135]),
  array([135]),
  array([135])],
 [24,
  24,
  28,
  66,
  67,
  102,
  107,
  111,
  140,
  162,
  171,
  174,
  174,
  187,
  205,
  205,
  205,
  205,
  205,
  205,
  205])

In [ ]:
import joblib

In [ ]:
filename = 'pac_20_91.sav'
joblib.dump(pac, filename)

['pac_20_91.sav']

[array([202]),
 array([202]),
 array([202]),
 array([202]),
 array([202]),
 array([203]),
 array([203]),
 array([203]),
 array([203]),
 array([204]),
 array([109]),
 array([111]),
 array([109]),
 array([189]),
 array([105]),
 array([141]),
 array([127]),
 array([135]),
 array([135])]

In [ ]:
from joblib import dump, load

In [ ]:
svm_sgd= load('/content/svm_sgd.sav') 

In [ ]:
pac= load('/content/pac.sav') 

In [ ]:
y_pred_svm = svm_sgd.predict(X_test)

In [ ]:
y_pred_pac = pac.predict(X_test)

In [ ]:
import pandas as pd

In [ ]:
my_submit = pd.DataFrame({'Expected': y_test, 'Predicted_svm': y_pred_svm,'Predicted_pac': y_pred_pac})
my_submit.head(20)

,Expected,Predicted_svm,Predicted_pac
0,0,0,0
1,0,131,131
2,0,0,0
3,0,0,0
4,0,0,0
5,1,1,1
6,1,1,1
7,1,1,1
8,1,1,1
9,1,1,1


In [ ]:
nn = pd.read_csv('/content/nn_predictions.csv', index_col='Unnamed: 0')

In [ ]:
predicted_nn = nn['Predicted_svm']

In [ ]:
my_submit['Predicted_nn'] = predicted_nn

In [ ]:
my_submit.sample(n=20)

,Expected,Predicted_svm,Predicted_pac,Predicted_nn
1026,174,174,174,174
936,165,165,165,165
187,30,30,30,30
1124,183,183,183,183
369,60,60,60,23
821,139,139,139,139
8,1,1,1,1
224,37,37,37,37
593,103,202,156,103
584,102,102,102,102


In [ ]:
my_submit.to_csv('/content/predictions.csv', index=True,)

,Unnamed: 0,Expected,Predicted_svm,Predicted_pac,Predicted_nn
0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,131.0,131.0,0.0
2,2.0,0.0,0.0,0.0,0.0
3,3.0,0.0,0.0,0.0,0.0
4,4.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...
1374,1374.0,204.0,109.0,109.0,93.0
1375,1375.0,204.0,109.0,109.0,67.0
1376,1376.0,204.0,204.0,204.0,204.0
1377,NaN,NaN,NaN,NaN,NaN



# CATBOOST


In [ ]:
# Code for recording audio from the browser
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode
import IPython
import uuid
from google.colab import output


class InvokeButton(object):
  def __init__(self, title, callback):
    self._title = title
    self._callback = callback

  def _repr_html_(self):
    from google.colab import output
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)

    template = """<button id="{callback_id}" style="cursor:pointer;background-color:#EEEEEE;border-color:#E0E0E0;padding:5px 15px;font-size:14px">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id)
    return html

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=3):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  with open('audio.wav','wb+') as f:
    f.write(b)
  return 'audio.wav'

In [ ]:
def classify(clf):
  print("Now recording for 3 seconds, say what you will...")
  record()
  os.system('ffmpeg -i audio.wav -ar 8000 -y audio.wav')
  audio, sr = librosa.load('audio.wav')
  mfcc = [librosa.feature.mfcc(audio[0:8000], sr, n_mfcc=80).reshape(80*16)]
  print(f"Audio recording complete, guess it is {clf.predict(mfcc)}")
  mfcc = [librosa.feature.mfcc(audio[8000:16000], sr, n_mfcc=80).reshape(80*16)]
  print(f"Audio recording complete, guess it is {clf.predict(mfcc)}")
  mfcc = [librosa.feature.mfcc(audio[16000:24000], sr, n_mfcc=80).reshape(80*16)]
  print(f"Audio recording complete, guess it is {clf.predict(mfcc)}")



In [ ]:
classify(clf)

Now recording for 3 seconds, say what you will...


<IPython.core.display.Javascript object>

Audio recording complete, guess it is [135]
Audio recording complete, guess it is [111]
Audio recording complete, guess it is [111]


In [ ]:
sgd_svm.predict([X_test_mfcc[-10]])

array([204])

In [ ]:
import soundfile as sf

In [ ]:
list_205_test = data_205_train.voiceprint.to_list()

In [ ]:
c=0
for i in list_205_test:
  print(np.array(i[1:-1].split(","), dtype=float))
  sf.write(str(c)+".wav",np.array(i[1:-1].split(","), dtype=float), 8000)
  c+=1

[ 0.00000000e+00  0.00000000e+00 -3.05175781e-05 ... -6.65466309e-01
 -6.37878418e-01 -6.95037842e-01]
[-0.00054932  0.00027466 -0.00143433 ...  0.0010376   0.00042725
  0.00482178]
[ 0.00527954 -0.00552368 -0.05566406 ... -0.00164795 -0.00085449
  0.00064087]
[ 0.12719727  0.10165405  0.11297607 ...  0.00549316 -0.0112915
 -0.01809692]
[ 0.25561523  0.06335449 -0.05722046 ... -0.08648682 -0.09136963
 -0.08868408]
[-0.13897705 -0.06365967  0.01046753 ... -0.00802612  0.00256348
  0.01446533]
[-0.06619263 -0.14401245  0.03790283 ...  0.14413452  0.11227417
  0.06335449]
[0.00283813 0.2348938  0.2925415  ... 0.00088501 0.00216675 0.00091553]
[ 0.18811035  0.19894409  0.15820312 ... -0.01281738 -0.04348755
 -0.05108643]
[-0.04199219 -0.01611328  0.03753662 ...  0.12030029  0.05706787
  0.01101685]
[0.23233032 0.23599243 0.1769104  ... 0.19812012 0.26464844 0.26062012]
[0.2947998  0.25250244 0.1809082  ... 0.00390625 0.00772095 0.0105896 ]
[-0.0609436  -0.01013184 -0.08877563 ... -0.001342

In [ ]:
IPython.display.Audio('11.wav')

CNN # New Section